<a href="https://colab.research.google.com/github/cccg8105/notebook_AD/blob/deep_learning/deep_learning/red_convolucional/RNC_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Red Neuronal Convolucional

## Importacion de repositorio con imágenes

In [1]:
!git clone https://github.com/cccg8105/deeplearning-az.git

from google.colab import drive
drive.mount('/content/drive')

Cloning into 'deeplearning-az'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 10185 (delta 48), reused 54 (delta 24), pack-reused 10096
Receiving objects: 100% (10185/10185), 237.00 MiB | 35.65 MiB/s, done.
Resolving deltas: 100% (73/73), done.
Checking out files: 100% (10113/10113), done.
Mounted at /content/drive


## Instalación de keras

In [2]:
pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.4.3)


## Creación del modelo

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

classifier = Sequential()

tamanioFiltro = 32
tamanioKernel = (3, 3)
funcionActivacion = "relu"
ventanaEntrada = (64, 64, 3)

classifier.add(Conv2D(filters = tamanioFiltro, kernel_size = tamanioKernel, 
                      input_shape = ventanaEntrada, activation = funcionActivacion))
# Max Pooling
# Obtiene el valor maximo de la ventana
tamanioPool = (2, 2)
classifier.add(MaxPooling2D(pool_size = tamanioPool))

# segunda capa de convolución y max pooling
funcionActivacion2 = "relu"
classifier.add(Conv2D(filters = tamanioFiltro, kernel_size = tamanioKernel, activation = funcionActivacion2))
classifier.add(MaxPooling2D(pool_size = tamanioPool))

# Flattening
classifier.add(Flatten())

# Full Connection
nodosEntrada = 128
nodosSalida = 1
funcionActivacion3 = "relu"
funcionActivacionFinal = "sigmoid"
classifier.add(Dense(units = nodosEntrada, activation = funcionActivacion3))
classifier.add(Dense(units = nodosSalida, activation = funcionActivacionFinal))

## Compilación 

In [4]:
optimizador = "adam"
funcionPerdida = "binary_crossentropy"
metricas = ["accuracy"]
classifier.compile(optimizer = optimizador, loss = funcionPerdida, metrics = metricas)

## Entrenamiento 

In [6]:
from keras.preprocessing.image import ImageDataGenerator

# modifica las imágenes para el entrenamiento
train_datagen = ImageDataGenerator(
        rescale=1./255, # transforma los pixeles
        shear_range=0.2, # mueve los pixeles
        zoom_range=0.2, # modifica el zoom
        horizontal_flip=True, 
        ) # rota la imagen

# modifica las imágenes para las pruebas
test_datagen = ImageDataGenerator(rescale=1./255)

# se configuran los datos de entrenamiento 
tamanioVentanaEntrada = (64, 64) # depende de la variable ventanaEntrada
tamanioLote = 32 # cantidad de imágenes que se van a procesar
modoClase = 'binary' # Solo se cuentan con dos categorias

training_dataset = train_datagen.flow_from_directory('/content/deeplearning-az/datasets/Part 2 - Convolutional Neural Networks (CNN)/dataset/training_set',
                                                    target_size=tamanioVentanaEntrada,
                                                    batch_size=tamanioLote,
                                                    class_mode=modoClase)

testing_dataset = test_datagen.flow_from_directory('/content/deeplearning-az/datasets/Part 2 - Convolutional Neural Networks (CNN)/dataset/test_set',
                                                target_size=tamanioVentanaEntrada,
                                                batch_size=tamanioLote,
                                                class_mode=modoClase)
# Entrenamiento 
pasos_x_epoca = 250 # muestras a tomar por época
epocas = 25 # cantidad de épocas
pasos_validacion = 2000 # cantidad de imágenes de prueba 
classifier.fit_generator(training_dataset,
                        steps_per_epoch=pasos_x_epoca,
                        epochs=epocas,
                        validation_data=testing_dataset,
                        validation_steps=pasos_validacion)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
250/250 [==============================] - 69s 276ms/step - loss: 0.6138 - accuracy: 0.6612 - val_loss: 0.5770 - val_accuracy: 0.7035
Epoch 2/25
250/250 [==============================] - 57s 230ms/step - loss: 0.5666 - accuracy: 0.7034
Epoch 3/25
250/250 [==============================] - 58s 231ms/step - loss: 0.5369 - accuracy: 0.7315
Epoch 4/25
250/250 [==============================] - 57s 229ms/step - loss: 0.5131 - accuracy: 0.7454
Epoch 5/25
250/250 [==============================] - 57s 230ms/step - loss: 0.4862 - accuracy: 0.7638
Epoch 6/25
250/250 [==============================] - 58s 231ms/step - loss: 0.4690 - accuracy: 0.7759
Epoch 7/25
250/250 [==============================] - 58s 231ms/step - loss: 0.4518 - accuracy: 0.7878
Epoch 8/25
250/250 [==============================] - 58s 231ms/step - loss: 0.4453 - accuracy: 0.7897
Epoch 9/25
250/250 [==============================

## Detectar objetos en imágenes

In [7]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/deeplearning-az/datasets/Part 2 - Convolutional Neural Networks (CNN)/dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_dataset.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

print(prediction)

dog
